In [2]:
from langchain_groq import ChatGroq

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY =  os.getenv("Groq_API_Key")

llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile"  
)


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data=loader.load().pop.page_content
print(page_data)

AttributeError: 'builtin_function_or_method' object has no attribute 'page_content'